In [1]:
# All codes which are done for setting up the server (Setting up the server environment, Installation of model dependencies and etc)

#########################################################################
# OCR part
#########################################################################
!sudo apt install tesseract-ocr


#########################################################################
# Object Detection part
#########################################################################
# install dependencies
def install():
  !pip install Flask
  !pip install flask-ngrok
  !pip install -U torch torchvision cython
  !pip install -U 'git+https://github.com/facebookresearch/fvcore.git' 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
  !git clone https://github.com/facebookresearch/detectron2 detectron2_repo
  !pip install -e detectron2_repo

install()

# After install
# You may need to restart your runtime prior to this, to let your installation take effect
# Some basic setup
# Setup detectron2 logger
import torch, torchvision
torch.__version__
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import cv2
import random
from google.colab.patches import cv2_imshow
from collections import OrderedDict

# import some common detectron2 utilities
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

# make pretrained model object
predictor = None
cfg = None
def setup():
  global cfg
  global predictor
  cfg = get_cfg()
  cfg.merge_from_file("./detectron2_repo/configs/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
  cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set threshold for this model
  # Find a model from detectron2's model zoo. You can either use the https://dl.fbaipublicfiles.... url, or use the following shorthand
  cfg.MODEL.WEIGHTS = "detectron2://COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x/137849600/model_final_f10217.pkl"
  predictor = DefaultPredictor(cfg)

setup()


#########################################################################
# TTS part
#########################################################################
# def install_tts():
#   # Clone git repo
#   !git clone https://github.com/CorentinJ/Real-Time-Voice-Cloning.git
#   %cd Real-Time-Voice-Cloning/
#   # Install dependencies
#   !pip install -q -r requirements.txt
#   !apt-get install -qq libportaudio2
#   # Download dataset
#   !gdown https://drive.google.com/uc?id=1n1sPXvT34yXFLT47QZA6FIRGrwMeSsZc
#   !unzip pretrained.zip
#   !gdown https://drive.google.com/uc?id=1eZNxBYyf9Rg7i7wwSpIyD91uaxkGqsPl

# install_tts()
  
# # Code for recording audio from the browser

# from IPython.display import Javascript
# from google.colab import output
# from base64 import b64decode
# import IPython
# import uuid
# from google.colab import output
# from IPython.display import Audio
# from IPython.utils import io
# from synthesizer.inference import Synthesizer
# from encoder import inference as encoder
# from vocoder import inference as vocoder
# from pathlib import Path
# import numpy as np
# import librosa
# synthesizer = None
# def setup_tts():
#   global synthesizer
#   print("inside set up")
#   encoder_weights = Path("encoder/saved_models/pretrained.pt")
#   vocoder_weights = Path("vocoder/saved_models/pretrained/pretrained.pt")
#   syn_dir = Path("synthesizer/saved_models/logs-pretrained/taco_pretrained")
#   encoder.load_model(encoder_weights)
#   synthesizer = Synthesizer(syn_dir)
#   vocoder.load_model(vocoder_weights)
# setup_tts()


#########################################################################
# Image Captioning part
#########################################################################
#실제로 할 때는 url 관련된 부분 지우고, IMAGE_FILE 변경, replace 경로 변경

import os
import sys

from tempfile import mkstemp
from shutil import move
from os import fdopen, remove


def replace(file_path, pattern, subst):
    #Create temp file
    fh, abs_path = mkstemp()
    with fdopen(fh,'w') as new_file:
        with open(file_path) as old_file:
            for line in old_file:
                new_file.write(line.replace(pattern, subst))
            old_file.close()
        new_file.close()
    #Remove original file
    remove(file_path)
    #Move new file
    move(abs_path, file_path)

def modify_files():
    OLD_CHECKPOINT_FILE = "/content/Pretrained-Show-and-Tell-model/model.ckpt-2000000"
    NEW_CHECKPOINT_FILE = "/content/Pretrained-Show-and-Tell-model/model2.ckpt-2000000"

    import tensorflow as tf
    vars_to_rename = {
        "lstm/basic_lstm_cell/weights": "lstm/basic_lstm_cell/kernel",
        "lstm/basic_lstm_cell/biases": "lstm/basic_lstm_cell/bias",
    }
    new_checkpoint_vars = {}
    reader = tf.train.NewCheckpointReader(OLD_CHECKPOINT_FILE)
    for old_name in reader.get_variable_to_shape_map():
        if old_name in vars_to_rename:
            new_name = vars_to_rename[old_name]
    else:
        new_name = old_name
    new_checkpoint_vars[new_name] = tf.Variable(reader.get_tensor(old_name))

    init = tf.global_variables_initializer()
    saver = tf.train.Saver(new_checkpoint_vars)

    with tf.Session() as sess:
        sess.run(init)
        saver.save(sess, NEW_CHECKPOINT_FILE)

def setup_img_captioning():
    # Pre-trained model git repo.
    %cd /content
    !git clone https://github.com/KranthiGV/Pretrained-Show-and-Tell-model.git

    # im2txt git repo.
    !git init img_caption
    %cd img_caption
    %ls .
    !git config core.sparseCheckout true
    !git remote add -f origin https://github.com/tensorflow/models.git
    !echo "research/im2txt" >> .git/info/sparse-checkout
    !git pull origin master

    %cd /content
    # download packages, libraries
    !pip install virtualenv
    !virtualenv venv
    !virtualenv -p /usr/bin/python2.7 --distribute temp-python
    # os.system("virtualenv -p /usr/bin/python2.7 --distribute temp-python")
    !source temp-python/bin/activate
    # !python -c 'import sys; print(sys.version_info[:])'
    !sudo apt autoremove
    !sudo apt-get install g++ unzip zip
    !sudo apt autoremove
    !sudo apt-get install openjdk-11-jdk
    !wget https://github.com/bazelbuild/bazel/releases/download/1.0.0/bazel-1.0.0-installer-linux-x86_64.sh
    !chmod +x bazel-1.0.0-installer-linux-x86_64.sh
    !./bazel-1.0.0-installer-linux-x86_64.sh --user
    os.environ['PATH'] += ':/root/bin'
    !pip install tensorflow==1.0.0
    !pip install numpy
    !pip install nltk
    !python -m nltk.downloader punkt

    !pip install requests

    # download checkpoint (2M iteration)
    from google_drive_downloader import GoogleDriveDownloader as gdd

    gdd.download_file_from_google_drive(file_id='0B3laN3vvvSD2WWxuR3VRQzhycWM',
                                        dest_path='/content/Pretrained-Show-and-Tell-model/model.ckpt-1000000.data-00000-of-00001')
    gdd.download_file_from_google_drive(file_id='0B3laN3vvvSD2T1RPeDA5djJ6bFE',
                                        dest_path='/content/Pretrained-Show-and-Tell-model/model.ckpt-2000000.data-00000-of-00001')
    
    # solved mismatch problem of variable name in tensorflow graph
    replace("/content/img_caption/research/im2txt/im2txt/inference_utils/caption_generator.py"
    , '[:-self.beam_size]', '')

    # modify_files()
    replace("/content/img_caption/research/im2txt/im2txt/run_inference.py", '      for i, caption in enumerate(captions):', '      f=open("/result.txt", mode="w+", encoding="utf-8")')
    replace("/content/img_caption/research/im2txt/im2txt/run_inference.py", '        sentence = [vocab.id_to_word(w) for w in caption.sentence[1:-1]]', '      sentence = [vocab.id_to_word(w) for w in captions[0].sentence[1:-1]]')
    replace("/content/img_caption/research/im2txt/im2txt/run_inference.py", '        sentence = " ".join(sentence)', '      sentence = " ".join(sentence)\n      f.write(sentence)\n      f.close()')
    replace("/content/img_caption/research/im2txt/im2txt/run_inference.py", '        print("  %d) %s (p=%f)" % (i, sentence, math.exp(caption.logprob)))', '      print("  %d) %s (p=%f)" % (0, sentence, math.exp(captions[0].logprob)))')


setup_img_captioning()

Reading package lists... Done
Building dependency tree       
Reading state information... Done
tesseract-ocr is already the newest version (4.00~git2288-10f4998a-2).
The following package was automatically installed and is no longer required:
  libnvidia-common-430
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 32 not upgraded.
Requirement already up-to-date: torch in /usr/local/lib/python3.6/dist-packages (1.3.1)
Requirement already up-to-date: torchvision in /usr/local/lib/python3.6/dist-packages (0.4.2)
Requirement already up-to-date: cython in /usr/local/lib/python3.6/dist-packages (0.29.14)
  Cloning https://github.com/facebookresearch/fvcore.git to /tmp/pip-req-build-apab0j8e
  Running command git clone -q https://github.com/facebookresearch/fvcore.git /tmp/pip-req-build-apab0j8e
  Cloning https://github.com/cocodataset/cocoapi.git to /tmp/pip-req-build-ibqcxtbe
  Running command git clone -q https://github.com/cocodataset/cocoapi.git /tmp

model_final_f10217.pkl: 178MB [00:11, 16.1MB/s]                           


/content
Cloning into 'Pretrained-Show-and-Tell-model'...
remote: Enumerating objects: 49, done.
remote: Total 49 (delta 0), reused 0 (delta 0), pack-reused 49
Unpacking objects: 100% (49/49), done.
Initialized empty Git repository in /content/img_caption/.git/
/content/img_caption
Updating origin
remote: Enumerating objects: 896, done.
remote: Counting objects: 100% (896/896), done.
remote: Compressing objects: 100% (527/527), done.
remote: Total 32953 (delta 483), reused 604 (delta 368), pack-reused 32057
Receiving objects: 100% (32953/32953), 512.26 MiB | 35.03 MiB/s, done.
Resolving deltas: 100% (20792/20792), done.
From https://github.com/tensorflow/models
 * [new branch]      BarretZoph-patch-1      -> origin/BarretZoph-patch-1
 * [new branch]      BarretZoph-patch-2      -> origin/BarretZoph-patch-2
 * [new branch]      MarkDaoust-patch-1      -> origin/MarkDaoust-patch-1
 * [new branch]      YknZhu-patch-1          -> origin/YknZhu-patch-1
 * [new branch]      YknZhu-patch-2   

In [2]:
# the code executed when server is running 
#########################################################################
# Object Detection part
#########################################################################
def inferrence(im, mode='labels'):
  outputs = predictor(im)

  #this create text labels from int labels
  def _create_text_labels(classes, scores, class_names):
      """
      Args:
          classes (list[int] or None):
          scores (list[float] or None):
          class_names (list[str] or None):

      Returns:
          list[str] or None
      """
      # we take items over score_threshold
      score_threshold = 0.8
      labels = None
      if classes is not None and class_names is not None and len(class_names) > 1:
          labels = [class_names[i] for i in classes]
      if scores is not None:
          if labels is None:
              labels = ["{:.0f}%".format(s * 100) for s in scores]
          else:
              labels = ["{} {:.0f}%".format(l, s * 100) for l, s in zip(labels, scores) if score_threshold >= 0.8]
      return labels

  # used like this in 'draw_instance_predictions(self, predictions)' (this 'self' is a Visualizer object)
  # scores = predictions.scores if predictions.has("scores") else None
  # classes = predictions.pred_classes if predictions.has("pred_classes") else None
  # labels = _create_text_labels(classes, scores, self.metadata.get("thing_classes", None))

  v = Visualizer(im[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)

  # We can use `Visualizer` to draw the predictions on the image.

  if mode == 'segmentation':
    v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    cv2_imshow(v.get_image()[:, :, ::-1])
  elif mode == 'labels':
    # variable 'labels' is text labels included in image
    predictions = outputs["instances"].to("cpu")
    scores = predictions.scores if predictions.has("scores") else None
    classes = predictions.pred_classes if predictions.has("pred_classes") else None
    labels = _create_text_labels(classes, scores, v.metadata.get("thing_classes", None))
    labels = [x.split()[0] for x in labels]
    labels = list(OrderedDict((element, None) for element in labels))
    return labels

#########################################################################
# TTS part
#########################################################################
def inferrence_tts(text):
  print("Audio synthesis with latest record")
  in_fpath = Path("audio.wav")
  reprocessed_wav = encoder.preprocess_wav(in_fpath)
  original_wav, sampling_rate = librosa.load(in_fpath)
  preprocessed_wav = encoder.preprocess_wav(original_wav, sampling_rate)
  embed = encoder.embed_utterance(preprocessed_wav)
  print("Synthesizing new audio...")
  with io.capture_output() as captured:
    specs = synthesizer.synthesize_spectrograms([text], [embed])
  generated_wav = vocoder.infer_waveform(specs[0])
  generated_wav = np.pad(generated_wav, (0, synthesizer.sample_rate), mode="constant")
  # display(Audio(generated_wav, rate=synthesizer.sample_rate))
  #librosa.output.write_wav("./result.wav", generated_wav, synthesizer.sample_rate)
  import numpy as np
  from scipy.io.wavfile import write
  write('first_sine_wave.wav', sps, generated_wav)
  return

#########################################################################
# OCR part
#########################################################################
def inferrence_ocr(im_path):
  import sys
  print("inside inference")
  os.system('tesseract '+im_path+' output')

#########################################################################
# Image Captioning part
#########################################################################
import subprocess as sp
from requests import get

def download_img(url):
  %cd /content
  download(url,"hello.jpg")

def download(url, file_name):
    with open(file_name, "wb") as file:
        response = get(url)
        file.write(response.content)
        file.close()

def run_img_captioning():
    CHECKPOINT_PATH="/content/Pretrained-Show-and-Tell-model/model.ckpt-2000000";
    VOCAB_FILE="/content/Pretrained-Show-and-Tell-model/word_counts.txt";
    IMAGE_FILE="/content/pics/hello.jpg";

    if os.path.exists('/result.txt'):
        os.remove('/result.txt')

    %cd /content/img_caption/research/im2txt
    !bazel build -c opt im2txt/run_inference

    !bazel-bin/im2txt/run_inference \
        --checkpoint_path=$CHECKPOINT_PATH \
        --vocab_file=$VOCAB_FILE \
        --input_files=$IMAGE_FILE

    g=open('/result.txt')
    line=g.readline()
    # print(line)
    g.close()
    return line

#code for actually running server
from flask import Flask
from flask import request
from flask import jsonify
from flask import make_response
from flask import send_from_directory
import os
from flask_ngrok import run_with_ngrok
app = Flask(__name__)
UPLOAD_FOLDER = './pics'
if not os.path.exists(UPLOAD_FOLDER):
  os.mkdir(UPLOAD_FOLDER)
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER
run_with_ngrok(app)   #starts ngrok when the app is run

#server code which handles request for object detection
@app.route('/picture', methods=['POST', 'GET'])
def upload_file():
    if request.method == 'POST':
        file = request.files['file']
        if file:
            filename = "hello.jpg"
            file.save(os.path.join(app.config['UPLOAD_FOLDER'], filename))
            im = cv2.imread("./pics/hello.jpg")
            result = inferrence(im)
            print(result)
            text = ""
            if len(result) == 0:
              text = "Unknown object"
            else:
              text = ', '.join(result)
            print(text)
            return jsonify({'result':text})

#server code which handles request for OCR
@app.route('/swipeDown', methods=['POST', 'GET'])
def upload_f():
    if request.method == 'POST':
        file = request.files['file']
        if file:
            print("in OCR")
            filename = "hello.jpg"
            file.save(os.path.join(app.config['UPLOAD_FOLDER'], filename))
            inferrence_ocr("./pics/hello.jpg")
            with open('./output.txt', 'r') as content_file:
              content = content_file.read()
            text = ""
            if len(content) == 0:
              text = "text not detected"
            else:
              text = content
            print(text)
        return jsonify({'result':text})

#server code which handles request for Image Captioning 
@app.route('/swipeUp', methods=['POST', 'GET'])
def upload_file2():
    if request.method == 'POST':
      file = request.files['file']
      if file:
          filename = "hello.jpg"
          file.save(os.path.join(app.config['UPLOAD_FOLDER'], filename))
          im = cv2.imread("/content/pics/hello.jpg")
          result = run_img_captioning()
          print(result)
          text = ""
          if len(result) == 0:
            text = "Unknown object"
          else:
            text = result
          print(text)
          return jsonify({'result':text})

app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://e220492d.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [01/Dec/2019 14:44:37] "POST /picture HTTP/1.1" 200 -


['mouse', 'keyboard', 'tv']
mouse, keyboard, tv


127.0.0.1 - - [01/Dec/2019 14:44:45] "POST /picture HTTP/1.1" 200 -


['chair', 'tv', 'laptop', 'cup', 'mouse', 'bird', 'keyboard', 'dining']
chair, tv, laptop, cup, mouse, bird, keyboard, dining
